# D&D Throw Simulation

Try to assess the expected returns for most spells available in D&D 5e, running basic probabilities, etc.

## Key Functions & Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

#Standard Params
size = 1000 #simulation size

In [2]:
def rollDice(dType):
    return np.random.randint(1, dType+1)

def abilityCheck(rollType=0):
    #rollType as 0 (straight), +1 (advantage), -1 (disadvantage)
    switcher = {
        0: rollDice(20),
        1: max([rollDice(20),rollDice(20)]),
        -1: min([rollDice(20),rollDice(20)])
    }
    return switcher.get(rollType,"Wrong Value")

## Roll Stats V2

In [3]:
stat = [rollDice(6) for i in range (0,4)]
stat

[5, 2, 1, 5]

In [4]:
def rollAbilityStat():
    stat = [rollDice(6) for i in range (0,4)]
    return sum(sorted(stat)[-3:])

In [6]:
throws = [rollAbilityStat() for i in range(0, size)]

throws = pd.Series(throws)
throws.describe()

count    1000.000000
mean       12.194000
std         2.854911
min         4.000000
25%        10.000000
50%        12.000000
75%        14.000000
max        18.000000
dtype: float64

In [5]:
[rollAbilityStat() for i in range (0,6)]

[17, 16, 12, 16, 11, 6]

## Ability Check Simulator

In [4]:
#Simulating an attack ability check
rollType = 0 #Simulating Advantage roll modifier. 0:straight, 1:advantage, -1:disadvantage
skillModifier = 4

throws = [abilityCheck(rollType) + skillModifier for i in range(0, size)]
throws = pd.Series(throws)
throws.describe()

count    1000.000000
mean       14.342000
std         5.708908
min         5.000000
25%        10.000000
50%        14.000000
75%        19.000000
max        24.000000
dtype: float64

In [8]:
#evaluating success rates against AC/DC
armorClass = 20

successRate = throws[throws >= armorClass].count() / throws.count()

print('Success Rate at AC {} : {:0.2%}'.format(armorClass, successRate))

Success Rate at AC 20 : 23.70%


## Spell & Damage Simulator

In [150]:
#Simulating 1k 1D6+4 attacks to evaluate the damage
dNumber = 1 #amount of dice to throw for each spell
dType = 6 #Die type
modifier = 4 #damage modifier

throws = pd.Series([sum([rollDice(dType) for i in range(0, dNumber)] + [modifier]) for i in range(0, size)])

print("Attack Stats for {}D{}+{} \n".format(dNumber, dType, modifier))
print(throws.describe())

Attack Stats for 1D6+4 

count    10000.000000
mean         7.502500
std          1.717873
min          5.000000
25%          6.000000
50%          7.000000
75%          9.000000
max         10.000000
dtype: float64


## Evaluating Ability Scores

Monte-Carloing the shit out of it

In [102]:
#utils

#abilities
abilities = [ "STR", "DEX", "CON", "INT", "WIS", "CHA"]

#Mara's current stat block
mara = {'STR': 7, 'DEX': 18, 'CON': 10, 'INT': 8, 'WIS': 9, 'CHA': 18}

En gros je voudrais en faire mille et regarder combien sont meilleurs. En fait ce qui est compliqué c'est de définir "meilleur". Je suppose que c'est, à la base, la probabilité d'avoir un ou plusieurs scores supérieurs. 

Pour chaque bloc que je génère, je compare les stats avec celles de Mara et je lui donne un score ? Ce qui m'intéresse c'est en gros de savoir le nombre de scores positifs. 

In [103]:
#rolling a single stat block
def rollStatBlock():
    attr = [ "STR", "DEX", "CON", "INT", "WIS", "CHA"]
    
    #2D6+5
    dNumber = 2
    dType = 6
    modifier = 5

    statblock = {var : sum([rollDice(dType) for i in range(0, dNumber)] + [modifier]) for var in attr}

    return statblock

randStatBlock = rollStatBlock()
randStatBlock

{'STR': 12, 'DEX': 11, 'CON': 12, 'INT': 11, 'WIS': 11, 'CHA': 8}

In [106]:
#rolling a series of stats without attributing them

def rollAbilities():
    #A series of 6 sums of 2D6+5
    throws = 6 
    dNumber = 2
    dType = 6
    modifier = 5

    stats = [sum([rollDice(dType) for i in range(0, dNumber)] + [modifier]) for i in range(0, throws)]

    return stats

#simple algorithm to assign randomized rolls to character scores
def assignScores(series):
    series.sort(reverse = True) #sorting rolls from highest to lowest
    abilities = ["DEX", "CHA", "CON", "WIS", "INT", "STR"] #character attributes are sorted by priority
    assignedScores = dict(zip(abilities, series)) #scores assigned on priority order

    return assignedScores


randScores = rollAbilities()
scores = assignScores(randScores)

scores

{'DEX': 16, 'CHA': 13, 'CON': 12, 'WIS': 11, 'INT': 11, 'STR': 11}

Je transforme ça en un dict contenant des listes plutôt que des entiers.

In [139]:
size = 1000 

#def simulateAbilities(size):

scoreComparison = {}

for i in range(0, size):
    randScores = rollAbilities()
    scores = assignScores(randScores)
    
    for ability in scores:
        if i == 0:
            scoreComparison[ability] = [scores[ability] - mara[ability]]
        else:
            scoreComparison[ability].append(scores[ability] - mara[ability])


In [140]:
#I'm going to append list and pull a simple description of this, see what this teaches me.
import itertools 
total = list(itertools.chain(*[scoreComparison[ability] for ability in scoreComparison]))
pd.Series(total).describe()

count    6000.000000
mean        0.319333
std         3.178581
min        -9.000000
25%        -3.000000
50%         1.000000
75%         3.000000
max         7.000000
dtype: float64

So the learning here. If I were to reroll, in average, I have a .3 improvement on every score.

In [142]:
pd.Series(scoreComparison['DEX']).describe()

count    1000.000000
mean       -2.999000
std         1.388858
min        -8.000000
25%        -4.000000
50%        -3.000000
75%        -2.000000
max        -1.000000
dtype: float64